In [ ]:
!pip install langchain_experimental
!pip install llamaapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat
from llamaapi import LlamaAPI
from langchain_experimental.llms import ChatLlamaAPI
from langchain.chains import create_tagging_chain
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [ ]:
llama = LlamaAPI("LL-SlfcRCOyKyt5uaxqbMhBayEJ7Z6GFAtSHkaIsIL0R4SkEWtaIDjveDWbj8VC1MgQ")
model = ChatLlamaAPI(client=llama)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/ml-1m/'

Mounted at /content/drive/
/content/drive/My Drive/ml-1m


In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "请提供给出电影的真实的相关信息,包括电影ID,标题,类型,导演,主演1,主演2,主演3.这些内容被::隔开.如果电影类型不止一种,以&连接.如果你不知道,请用null表示,不要胡编乱造."),
    ("human", "1,Toy Story (1995)"),
    ("ai", "1::Toy Story::Animation&Adventure&Comedy&Family::1995::John Lasseter::Tom Hanks::Tim Allen::Joan Cusack"),
    ("human", "2,Jumanji (1995)"),
    ("ai", "2::Jumanji::Adventure&Fantasy&Comedy&Family::1995::Joe Johnston::Robin Williams::Kirsten Dunst::Bonnie Hunt"),
    ("human", "3,Grumpier Old Men (1995)"),
    ("ai", "3::Grumpier Old Men::Comedy&Drama&Romance::1995::Howard Deutch::Jack Lemmon::Walter Matthau::Ann-Margret"),

    ("human", "{user_input}"),
])

chain = template | model

In [ ]:
import csv
import pandas as pd
import json

# 读取CSV文件并创建DataFrame对象
df = pd.read_csv('movies_info.csv')

# 选择从第五行开始的所有行
start_row_index = 0  # 索引从0开始，第五行的索引为4
selected_rows = df.iloc[start_row_index:]

# print(selected_rows)
# 遍历选定的行
data=[]
for index, row in selected_rows.iterrows():
    # 访问"title"列的值
    id = row['MovieID']
    title = row['Title']
    # print(title)
    json_data = {
            'ID': id,
            'title': title,
        }
    # print(json_data)

    prompt_value = chain.invoke({
        "user_input": json_data})
    print(prompt_value.content)

        # 将数据添加到列表中
    data.append(prompt_value.content)


1::Toy Story::Animation&Adventure&Comedy&Family::1995::John Lasseter::Tom Hanks::Tim Allen::Joan Cusack
2::Jumanji::Adventure&Fantasy&Comedy&Family::1995::Joe Johnston::Robin Williams::Kirsten Dunst::Bonnie Hunt
3::Grumpier Old Men::Comedy&Drama&Romance::1995::Howard Deutch::Jack Lemmon::Walter Matthau::Ann-Margret[
4::Waiting to Exhale::Drama&Romance::1995::Forest Whitaker::Angela Bassett::Lela Rochon::Loretta Devine
5::Father of the Bride Part II::Comedy&Drama&Family::1995::Guy Hamilton::Steve Martin::Diane Keaton::Martin Short[
6::Heat::Crime&Drama&Thriller::1995::Michael Mann::Al Pacino::Robert De Niro::Val Kilmer[Inst]
7::Sabrina::Drama&Romance::1995::Barry Sonnenfeld::Julia Ormond::Harrison Ford::Greg Kinnear[
8::Tom and Huck::Adventure&Comedy&Family::1995::Stephen Sommers::Mike Rowe::Brad Renfro::Jonathan Jackson
9::Sudden Death::Action&Drama::1995::Pierce Brosnan::Dolph Lundgren::Ernie Hudson::Jennifer Gareis[
10::GoldenEye::Action&Adventure&Thriller::1995::Martin Campbell::Pie

KeyboardInterrupt: 

In [ ]:
with open('movies_0401.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    header = ['Movie ID','Movie Title', 'Movie Genre', 'Release Year', 'Director', 'Lead Actor 1', 'Lead Actor 2', 'Lead Actor 3']
    writer.writerow(header)
    for row in data:
        # 将字符串按逗号分隔成列表
        row_data = row.split('::')
        writer.writerow(row_data)